<div class="alert alert-block alert-info">
<b>Sentence BERT를 이용한 내용 기반 국문 저널 추천 시스템</b>
</div>

# IV. Exponential Function + Weight 적용 모델

---
# Setting

In [278]:
# setting path
import sys
sys.path.append('..') # parent directory 경로 추가

from common import *
from my import *

%matplotlib inline
%config InlineBackend.figure_format='retina'

# random seed 고정 
import os, random
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
#     tf.random.set_seed(seed) # Tensorflow 사용시 
SEED = 777
set_seeds(SEED)

In [279]:
import os
import pickle
from tqdm import tqdm

import collections
from heapq import nlargest
from operator import itemgetter
from joblib import Parallel, delayed, parallel_backend

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt, Mecab

import math
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

---
# Train 데이터 관련 준비

In [260]:
# 전처리한 train 데이터(dataframe) 다시 읽어오기
train = pd.read_csv('./data/train.csv', index_col=0)
print(train.shape)

# Train y 
train_y = train['journal'] # Train 데이터 document id - journal
print(train_y.shape)

# Train 데이터셋 기준 각 document id - index 매핑 (데이터셋에서의 위치)
train_id_index = pd.Series(range(len(train.index)), index=train.index)
print(len(train_id_index))

# Train SBERT embedding npy 파일 다시 읽기
train_embed = np.load('./data/train_embed.npy')
print(train_embed.shape)

(92760, 7)
(92760,)
92760
(92760, 768)


In [92]:
# 저널명 - Index 매핑  (저널 고정 순서)
journ_order = train.groupby('journal').groups.keys()
journ_index = pd.Series(range(len(journ_order)), index=journ_order)
print(journ_index[:5])
print(journ_index[-5:])

journal
CRM연구                                                      0
Child Health Nursing Research                              1
Clinical and Experimental Reproductive Medicine            2
Clinics in Shoulder and Elbow                              3
Communications for Statistical Applications and Methods    4
dtype: int64
journal
해양환경안전학회지    263
혜화의학회지       264
화약ㆍ발파        265
환경영향평가       266
환경정책연구       267
dtype: int64


In [261]:
from sklearn.feature_extraction.text import CountVectorizer

# Journal-Title Matrix (JTM)
# journal별 title CountVectorize
train_journ_tt = train.groupby('journal')['title_nn'].apply(' '.join) # journal별로 모든 논문들 title text 합치기
count_vect_tt = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jtm = count_vect_tt.fit_transform(train_journ_tt) # sparse matrix
print(jtm.shape)

# csr matrix -> dataframe
# jtm_df = pd.DataFrame(jtm.toarray(), index=train_journ_tt.index, columns=count_vect_tt.get_feature_names_out())

# ================================================= #
# Journal-Keyword Matrix (JKM)
# journal별 keyword CountVectorize
train_journ_kw = train.groupby('journal')['keyword'].apply(' '.join) # journal별로 모든 논문들 keyword text 합치기
count_vect_kw = CountVectorizer(min_df=1, ngram_range=(1,1)) # unigram
jkm = count_vect_kw.fit_transform(train_journ_kw) # sparse matrix
print(jkm.shape)

# csr matrix -> dataframe
# jkm_df = pd.DataFrame(jkm.toarray(), index=train_journ_kw.index, columns=count_vect_kw.get_feature_names_out())

(268, 43041)
(268, 135402)


---
# Test 데이터 관련 준비

In [264]:
# 전처리한 test 데이터(dataframe) 다시 읽어오기
test = pd.read_csv('./data/test.csv', index_col=0)
print(test.shape)

# Test y
test_y = test['journal'] # Test 데이터 document id - journal
print(test_y.shape)

# Test 데이터셋 기준 document id - index 매핑 (데이터셋에서의 위치)
test_id_index = pd.Series(range(len(test.index)), index=test.index)
print(len(test_id_index))

# Test SBERT embedding npy 파일 다시 읽기
test_embed = np.load('./data/test_embed.npy')
print(test_embed.shape)

(10307, 7)
(10307,)
10307
(10307, 768)


---
# # Phase 1: 후보 저널 리스트

## 문서 vs. 문서 유사도 (S1)
- Test 데이터 사용
- Test 데이터의 SBERT 임베딩된 Abstract vs. Train 데이터의 SBERT 임베딩된 Abstract -> 코사인 유사도 계산

In [ ]:
# test document의 embedding vectors & train document의 embedding vectors 코사인 유사도 분석
from sklearn.metrics.pairwise import cosine_similarity

s1 = cosine_similarity(test_embed, train_embed) # test embed vs. train embed
print(s1.shape)
s1[:5]

(10307, 92760)


array([[0.5445481 , 0.33625972, 0.1614262 , ..., 0.3330929 , 0.557525  ,
        0.31247985],
       [0.6503759 , 0.45653108, 0.29908445, ..., 0.50919765, 0.6316433 ,
        0.48862016],
       [0.31182605, 0.34367323, 0.24949932, ..., 0.44575486, 0.24206194,
        0.45320866],
       [0.42790985, 0.48927057, 0.2149891 , ..., 0.47800317, 0.53168607,
        0.46760318],
       [0.46252072, 0.38382778, 0.40913084, ..., 0.26628563, 0.23586869,
        0.44414553]], dtype=float32)

---
## Customized Function | List of Journals (R)

- Exponential Function 적용 및 추가 가중치 부여
- 문서별 유사도 점수 계산 방법:
    - 코사인 유사도 0.7 = 70, 1.0 = 1000이 되도록 exponential function 적용
    - 저널별로 유사도 높은(>threshold) document 수를 세고, 각 해당 저널에 속하는 document별 exponential function 적용된 유사도에 "document 수"를 추가 가중치(weight)로 부여
        - 유사도 높은 document가 많은 저널일수록 더 중요 할 수 있으니, 저널의 유사도를 최종 유사도에 추가로 반영한 것
- 저널별 최종 점수 계산 방법:
    - 저널별로 위 유사도 점수를 평균(mean)

In [259]:
# 1. 각 train document가 속한 저널이 유사도 0.7 이상의 document가 몇 개인지 따라 weight 부여 (-> 기존 exponential function에 더하기)
# -> 다시 말해, 각 train document의 유사도 점수 뿐만 아니라, 현재 test document와 관련이 클것 같은 저널일수록, 저널의 document일수록, 그 만큼의 가중치가 더해짐
# 2. 그리고, 저널별 모든 해당되는 document의 weighted 유사도를 sum하는 것이 아닌 mean하여 최종 score 게산

import math
import collections
from heapq import nlargest
from operator import itemgetter
from joblib import Parallel, delayed, parallel_backend

def get_all_sim_journ(doc_sims, threshold=0.7):
    '''
    doc_sims : numpy array; 각 문서vs.문서 유사도 점수
    threshold : "높은 유사도"의 기준
    '''
    # weighted similarity score 계산
    doc_and_scores = dict(zip(train_id_index.index, doc_sims))
    journ_and_scores = list(zip(train_y.values, doc_sims))
    docs_over_th = list((dict(filter(lambda x: x[1] >= threshold, doc_and_scores.items())).keys()))
    journs_over_th = collections.Counter(train_y[docs_over_th].values)
    weight = [journs_over_th[journ_name] if journ_name in journs_over_th else 0 for journ_name in train_y.values]
    
    sim_weighted = ((0.135 * np.exp(0.0891 * (doc_sims*100))) + 0.5) + weight # document별 weighted similarity
    
    # 각 점수마다 해당 document id 붙이기 ([document id, weighted 유사도 점수] 형태로)
    sim_weighted = list(zip(train_id_index.index, sim_weighted)) # e.g. ('JAKO201610364779000', 0.5445481)
    
    # weighted 유사도 >= threshold
    #sim_weighted = list(filter(lambda x: x[1] >= threshold, sim_weighted))
    
    # [document id, 유사도 점수] -> 저널별 [journal 이름, 유사도 총 점수]
    sum_dict = collections.defaultdict(list)
    for doc_id, sim_score in sim_weighted:
        journ_name = train_y[doc_id] # 저널명 가져오기
        # sum_dict[journ_name] += sim_score
        sum_dict[journ_name].append(sim_score) # 저널별 유사도 점수 합치기
    
    mean_dict = {key: np.mean(values) for key, values in sum_dict.items()}
    
    # 총 유사도 점수가 높은 순서대로 저널 이름 정렬 (S-Sort)
    journ_sorted = list(dict(sorted(mean_dict.items(), key=itemgetter(1), reverse=True)).keys())
    
    return journ_sorted

# 추천 후보 저널 - list of journals (R)
workers = os.cpu_count() * 2
with parallel_backend(backend='loky', n_jobs=workers):
    r = list(Parallel()(delayed(get_all_sim_journ)(doc_sims) for doc_sims in tqdm(s1, position=0, leave=True)))

print(len(r))
print(r[0][:10])

100%|█████████████████████████████████████| 10307/10307 [08:38<00:00, 19.89it/s]


10307
['원예과학기술지', '시설원예ㆍ식물공장', '한국가금학회지', '한국응용곤충학회지', '농약과학회지', '한국환경생태학회지', 'Weed & Turfgrass Science', 'Journal of Animal Science and Technology', 'Restorative Dentistry and Endodontics', '대한환경공학회지']


In [163]:
import pickle

# 유사도 높은 journal 리스트 pickle로 저장
# with open('./data/r_exp_weighted.pkl', 'wb') as f:
#     pickle.dump(r, f)

# 저장한 pickle 다시 읽기
# with open('./data/r_exp_weighted.pkl', 'rb') as f:
#     r = pickle.load(f)

----
# # Phase 2: 후보 저널 재정렬
- 후보 저널 재정렬
    - 입력 문서 vs. 저널의 Title, Keyword 유사도 기반 (S2+S3)
    - 초록 외 키워느나 제목이 입력된 경우, 저널에 출판된 문서들(train)의 키워드와 일치도를 계산하여 추천 결과를 개선하기 위한 과정
    - 후보 저널 리스트업 된 것에 순위 sort하기 위한 과정

## 문서 vs. 저널 유사도 (S2+S3)
- Keyword 유사도 (S2) + Title 유사도 (S3)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# ================================================= #
# test document vs. train journal의 keyword 유사도 구하기 (S2)

# 각 Test 데이터의 문서별 Keyword Matrix 생성 (Test JKM)
test_doc_kw = test['keyword']
print(test_doc_kw.shape)

# document-keyword matrix 생성 (JKM 생성때 fit된 count vectorizer 사용)
test_doc_kw_mat = count_vect_kw.transform(test_doc_kw)
print(test_doc_kw_mat.shape)

# dataframe 형태로 확인
# d_kw_df = pd.DataFrame(d_kw_mat.toarray(), index=d_kw.index, columns=count_vect_kw.get_feature_names_out())

# keyword 사이의 document vs. journal 유사도 계산
s2 = cosine_similarity(test_doc_kw_mat, jkm)
print(s2.shape)

# ================================================= #
# test document vs. train journal의 title 유사도 구하기 (S3)

# 각 Test 데이터의 문서별 Title Matrix 생성 (Test JTM)
test_doc_tt = test['title_nn']
print(test_doc_tt.shape)

# document-title matrix 생성 (JTM 생성때 fit된 count vectorizer 사용)
test_doc_tt_mat = count_vect_tt.transform(test_doc_tt)
print(test_doc_tt_mat.shape)

# dataframe 형태로 확인
# d_tt_df = pd.DataFrame(d_tt_mat.toarray(), index=d_tt.index, columns=count_vect_tt.get_feature_names_out())

# title 사이의 document vs. journal 유사도 계산
s3 = cosine_similarity(test_doc_tt_mat, jtm)
print(s3.shape)

# ================================================= #
# Title 코사인 유사도 + Keyword 코사인 유사도 더하기 (S2+S3)
s23 = s2 + s3
print(s23.shape)
s23[:5]

(10307,)
(10307, 135402)
(10307, 268)
(10307,)
(10307, 43041)
(10307, 268)
(10307, 268)


array([[0.16510017, 0.09318388, 0.07643517, ..., 0.03647989, 0.12525107,
        0.07358115],
       [0.        , 0.        , 0.        , ..., 0.0032244 , 0.00759136,
        0.00520297],
       [0.1249516 , 0.06112054, 0.05013484, ..., 0.19781094, 0.13443356,
        0.11188206],
       [0.        , 0.        , 0.        , ..., 0.03224398, 0.01180878,
        0.00346865],
       [0.11973687, 0.12910096, 0.07206377, ..., 0.17196787, 0.17375783,
        0.1314451 ]])

## 추천 저널 리스트 재배치

**Note: 코드가 실행되는 곳은 아래 "Phase"의 모델링 부분에서 실행 됨**

In [ ]:
# 문서 vs. 저널 유사도(S2+S3)에 따른 후보 저널 재배치

from operator import itemgetter

# def resort_r(tt_kw_s, i):
#     # doc-journal별 유사도 점수에 journal 이름 붙이기 
#     journ_scores = list(zip(journ_index.index, tt_kw_s[i])) # journal 이름 붙이기
#     # 높은 유사도순으로 journal 정렬
#     journ_scores_sorted = sorted(journ_scores, key=lambda x: x[1], reverse=True) 
#     # journal이 기존 list of journals에 있는 journal이라면 포함 (정렬된 순서에 맞춰 새로운 R 생성)
#     journ_filtered = [journ for journ, sim in journ_scores_sorted if journ in top_k_journs[i]]
#     return journ_filtered

def resort_r(tt_kw_s, top_k_journs, i):
    # 각 doc의 journal별 title+keyword 유사도 점수에 journal 이름 붙이기 
    journ_scores = dict(zip(journ_index.index, tt_kw_s[i])) # journal 이름 붙이기
    # 높은 유사도순으로 journal 재배치 (기존 list of journals에 있는 저널만 포함) -> 새로운 R 생성
    journ_sorted = list(dict(sorted(filter(lambda x: x[0] in top_k_journs[i], journ_scores.items()), key=itemgetter(1), reverse=True)).keys())
    return journ_sorted

---
# # Phase 3: 후보 저널 추가
- Test 데이터셋 사용
- 후보 저널 추가
    - 저널 vs. 저널 유사도 (S4) 기반
    - Title, Keyword 유사도
    - 새로 추가할 수 있는, 상위 후보 저널과 유사한 후보 저널을 탐색

## 저널 vs. 저널 유사도 (S4) | 후보 저널 추가

**Note: 코드가 실행되는 곳은 아래 "Phase"의 모델링 부분에서 실행 됨**

In [343]:
import collections
from heapq import nlargest
from operator import itemgetter
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed, parallel_backend

# 저널 vs. 저널 유사도 (S4) 계산 및 후보 저널 추가
def find_other_sim_journ(cur_top_journs):
    '''
    # Description :
    # 추천 저널 후보 리스트에서, 맨 앞의 최상위 저널(R1) 1개에 대해 저널들과의 유사도를 분석 (저널 vs. 저널 유사도 분석)
    # 이후, 유사도 높은 저널부터 순차적으로 추천 저널 후보 리스트에 없는 저널을 찾기
    # 만약 찾았지만, 발견한 저널의 유사도 점수가 0보다 크지 않다면 (그 뒤 후보들의 유사도는 볼 필요 없이)
    # R1은 pass하고 다음 상위 저널(R2, R3, ... 순차적으로)로 같은 방법으로 탐색
    '''
    def if_not_in_r(x):
        # list of journals에 이미 있는지/없는지 확인
        if x[0] in cur_top_journs:
            return False
        else:
            return True
    
    # list of journals이 애초에 비어있는 경우
    if not len(cur_top_journs):
        # 빈 list of journals 그대로 반환
        return cur_top_journs
    
    # 추가 후보 저널 탐색
    add_journ_found = False
    for i in range(len(cur_top_journs)):
        # 현재 선택된 상위 저널
        best_journ = cur_top_journs[i] # 맨 앞의(최상위) 후보 저널부터 하나씩

        # # 해당 상위 저널과 다른 저널들의 title 유사도 구하기
        # best_j_tt = train[train['journal'] == best_journ].groupby('journal')['title_nn'].apply(' '.join) # 해당 상위 저널의 (모든 doc의) 전처리된 title text 모으기
        # best_j_tt_mat = count_vect_tt.transform(best_j_tt) # 상위 저널의 journal-title matrix 생성 (train JTM 생성시 fit한 count vectorizer로 transform)
        best_j_tt_mat = jtm[journ_index[best_journ]]
        best_tt_sim = cosine_similarity(best_j_tt_mat, jtm) # 상위 저널 JTM - train JTM 과의 title 코사인 유사도 (1, 268)

        # 해당 상위 저널과 다른 후보 저널들의 keyword 유사도 구하기
        # best_j_kw = train[train['journal'] == best_journ].groupby('journal')['keyword'].apply(' '.join) # 해당 상위 저널의 (모든 doc의) 전처리된 keyword text 모으기
        # best_j_kw_mat = count_vect_kw.transform(best_j_kw) # 상위 저널의 journal-keyword matrix 생성 (train JKM 생성시 fit한 count vectorizer로 transform)
        best_j_kw_mat = jkm[journ_index[best_journ]]
        best_kw_sim = cosine_similarity(best_j_kw_mat, jkm) # 상위 저널 JKM - train JKM 과의 keyword 코사인 유사도

        # 해당 후보 저널의 다른 저널들과의 최종 유사도 구하기
        s4 = best_tt_sim + best_kw_sim # Title 코사인 유사도 + Keyword 코사인 유사도 더하기
        s4 = s4[0] # 268개 각 저널별 유사도 점수; test document 1개씩 처리 중이기 때문에 [i]가 아닌 [0] indexing
        
        # 높은 유사도 순으로 후보 저널 정렬0
        best_jj_sim = sorted(dict(zip(journ_index.index, s4)).items(), key=itemgetter(1), reverse=True)

        # 그 중 기존 list of journals에 없는 저널 탐색 (유사도 높은 후보부터)
        new_journ_found = next(filter(if_not_in_r, best_jj_sim), None)
        if new_journ_found != None: # list of journals에 없는 후보 저널 1개를 찾았고,
            if new_journ_found[1] > 0: # 해당 저널의 유사도 점수가 0보다 크다면
                # 후보 저널 추가
                top_journs_added = cur_top_journs + [new_journ_found[0]] # list of journals에 추가
                add_journ_found = True
                return top_journs_added
            else: # 유사도 점수가 0이면, 다음 후보 저널로 탐색 (R2,R3,...)
                continue
        else: # list of journals에 없는 후보 저널이 없다면(모든 후보 저널이 이미 다 list of journals에 있다면)
            continue # 다음 후보 저널로 탐색 (R2,R3,...)
    
    # 모든 후보 저널을(R1,R2,R3,...) 다 탐색했지만, 적합한 후보 저널을 못 찾았다면
    if not add_journ_found:
        top_journs_added = cur_top_journs # 기존 list of journals 그대로 사용
        return top_journs_added

# # 모델링 및 평가
- Best model 사용
- Top-K 저널 추천 및 정확도 평가

In [ ]:
# 추천 모델 평가 지표 함수

# Micro Accuracy
def get_acc_micro(y_pred, y_true):
    return np.mean(np.array([1 if y_true[i] in y_pred[i] else 0 for i in range(len(y_true))])) 

# Macro Accuracy
def get_acc_macro(y_pred, y_true):
    global each_journ_ox, each_journ_mean_acc
    # 저널별로 맞은 것은 1, 틀린 것은 0으로 넣기
    each_journ_ox = collections.defaultdict(list)
    for journ_actual, journs_rec in list(zip(y_true, y_pred)):
        if journ_actual in journs_rec:
            each_journ_ox[journ_actual].append(1)
        else:
            each_journ_ox[journ_actual].append(0)
    # 저널별 평균 accuracy 계산
    each_journ_mean_acc = {key: np.mean(values) for key, values in each_journ_ox.items()} # dictionary comprehension
    # 저널별 평균 accuracy의 전체 평균 계산
    mean_of_mean_acc = np.mean(list(each_journ_mean_acc.values()))
    return mean_of_mean_acc

# MRR
def get_mrr(y_pred, y_true):
    # 각 예측별 reciprocal rank 구하기 (실제 저널이 추천 리스트에서 몇 번째에 위치하는지)
    recip_rank_computed = []
    for journ_actual, journs_rec in list(zip(y_true, y_pred)):
        try:
            # 추천 리스트에 있다면, 위치 번호에 역수 취하기 (앞에 나올수록 좋은 모델이니까)
            recip_rank = 1 / (journs_rec.index(journ_actual) + 1) # 1/(index + 1)
        except:
            # 추천 리스트에 없다면 0
            recip_rank = 0
        recip_rank_computed.append(recip_rank)
    # Mean Reciprocal Rank 계산
    mmr = np.mean(recip_rank_computed)
    return mmr

In [334]:
# 추천 저널 수(K) 설정
# Top-3, Top-5, Top-10, Top-15, Top-20

K_list = [3, 5, 10, 15, 20]

## Phase 1: Abs
- 바로 Top-K 저널 추출

In [287]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = False
tt_kw_s = None

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K
    first_n = K
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    # workers = -1
    # with parallel_backend(backend='threading', n_jobs=workers):
    #     top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    top_k_r = r_resort
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_1 = accuracy_results.copy()
print(acc_abstr_1)
acc_abstr_1

Current K: 3
Current K: 5
Current K: 10
Current K: 15
Current K: 20
           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.5195 0.6356  0.7541  0.8167  0.8546
macro_acc 0.4332 0.5544  0.6803  0.7532  0.8073
mrr       0.3879 0.4144  0.4305  0.4355  0.4376


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.5195,0.6356,0.7541,0.8167,0.8546
macro_acc,0.4332,0.5544,0.6803,0.7532,0.8073
mrr,0.3879,0.4144,0.4305,0.4355,0.4376


In [288]:
import pandas as pd

# 결과 csv 저장
acc_abstr_1.to_csv('./acc_model4/acc_abstr_1.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_1 = pd.read_csv('./acc_model4/acc_abstr_1.csv', index_col=0)
# acc_abstr_1

## Phase 1+2: Abs + Kw
- Top-K 저널 추출 후, 저널 재배치

In [281]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = True
tt_kw_s = s2

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K
    first_n = K
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    # workers = -1
    # with parallel_backend(backend='threading', n_jobs=workers):
    #     top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    top_k_r = r_resort
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_kw_1_2 = accuracy_results.copy()
print(acc_abstr_kw_1_2)
acc_abstr_kw_1_2

Current K: 3


100%|███████████████████████████████████| 10307/10307 [00:03<00:00, 2594.64it/s]


Current K: 5


100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2573.01it/s]


Current K: 10


...

100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2548.68it/s]


Current K: 20


100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2509.52it/s]

           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.5195 0.6356  0.7541  0.8167  0.8546
macro_acc 0.4332 0.5544  0.6803  0.7532  0.8073
mrr       0.4018 0.4327  0.4379  0.4327  0.4261


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.5195,0.6356,0.7541,0.8167,0.8546
macro_acc,0.4332,0.5544,0.6803,0.7532,0.8073
mrr,0.4018,0.4327,0.4379,0.4327,0.4261


In [282]:
import pandas as pd

# 결과 csv 저장
acc_abstr_kw_1_2.to_csv('./acc_model4/acc_abstr_kw_1_2.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_kw_1_2 = pd.read_csv('./acc_model4/acc_abstr_kw_1_2.csv', index_col=0)
# acc_abstr_kw_1_2

## Phase 1+2: Abs + Tt
- Top-K 저널 추출 후, 저널 재배치

In [284]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = True
tt_kw_s = s3

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K
    first_n = K
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    # workers = -1
    # with parallel_backend(backend='threading', n_jobs=workers):
    #     top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    top_k_r = r_resort
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_tt_1_2 = accuracy_results.copy()
print(acc_abstr_tt_1_2)
acc_abstr_tt_1_2

Current K: 3


100%|███████████████████████████████████| 10307/10307 [00:03<00:00, 2637.39it/s]


Current K: 5


100%|███████████████████████████████████| 10307/10307 [00:03<00:00, 2628.92it/s]


Current K: 10


...

100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2567.49it/s]


Current K: 20


100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2548.06it/s]

           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.5195 0.6356  0.7541  0.8167  0.8546
macro_acc 0.4332 0.5544  0.6803  0.7532  0.8073
mrr       0.3982 0.4270  0.4237  0.4124  0.4028


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.5195,0.6356,0.7541,0.8167,0.8546
macro_acc,0.4332,0.5544,0.6803,0.7532,0.8073
mrr,0.3982,0.4270,0.4237,0.4124,0.4028


In [285]:
import pandas as pd

# 결과 csv 저장
acc_abstr_tt_1_2.to_csv('./acc_model4/acc_abstr_tt_1_2.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_tt_1_2 = pd.read_csv('./acc_model4/acc_abstr_tt_1_2.csv', index_col=0)
# acc_abstr_tt_1_2

## Phase 1+2: Abs + Tt + Kw
- Top-K 저널 추출 후, 저널 재배치

In [278]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = True
tt_kw_s = s23

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K
    first_n = K
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    # workers = -1
    # with parallel_backend(backend='threading', n_jobs=workers):
    #     top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    top_k_r = r_resort
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_tt_kw_1_2 = accuracy_results.copy()
print(acc_abstr_tt_kw_1_2)
acc_abstr_tt_kw_1_2

Current K: 3


100%|███████████████████████████████████| 10307/10307 [00:03<00:00, 2582.86it/s]


Current K: 5


100%|███████████████████████████████████| 10307/10307 [00:03<00:00, 2602.96it/s]


Current K: 10


...

100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2571.84it/s]


Current K: 20


100%|███████████████████████████████████| 10307/10307 [00:04<00:00, 2563.38it/s]


           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.5195 0.6356  0.7541  0.8167  0.8546
macro_acc 0.4332 0.5544  0.6803  0.7532  0.8073
mrr       0.4095 0.4466  0.4602  0.4574  0.4538


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.5195,0.6356,0.7541,0.8167,0.8546
macro_acc,0.4332,0.5544,0.6803,0.7532,0.8073
mrr,0.4095,0.4466,0.4602,0.4574,0.4538


In [279]:
import pandas as pd

# 결과 csv 저장
acc_abstr_tt_kw_1_2.to_csv('./acc_model4/acc_abstr_tt_kw_1_2.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_tt_kw_1_2 = pd.read_csv('./acc_model4/acc_abstr_tt_kw_1_2.csv', index_col=0)
# acc_abstr_tt_kw_1_2

## Phase 1+3: Abs
- Top-(K-1) 저널 추출 후, 유사 저널 1개 추가

In [355]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = False
tt_kw_s = None

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K-1
    first_n = K-1
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    workers = -1
    with parallel_backend(backend='threading', n_jobs=workers):
        top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_1_3 = accuracy_results.copy()
print(acc_abstr_1_3)
acc_abstr_1_3

Current K: 3


100%|████████████████████████████████████| 10307/10307 [00:39<00:00, 261.81it/s]


Current K: 5


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 296.34it/s]


Current K: 10


...

100%|████████████████████████████████████| 10307/10307 [00:39<00:00, 263.81it/s]


Current K: 20


100%|████████████████████████████████████| 10307/10307 [00:39<00:00, 262.48it/s]


           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.4837 0.6185  0.7587  0.8211  0.8590
macro_acc 0.3961 0.5275  0.6775  0.7549  0.8086
mrr       0.3759 0.4110  0.4310  0.4358  0.4379


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.4837,0.6185,0.7587,0.8211,0.8590
macro_acc,0.3961,0.5275,0.6775,0.7549,0.8086
mrr,0.3759,0.4110,0.4310,0.4358,0.4379


In [356]:
import pandas as pd

# 결과 csv 저장
acc_abstr_1_3.to_csv('./acc_model4/acc_abstr_1_3.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_1_3 = pd.read_csv('./acc_model4/acc_abstr_1_3.csv', index_col=0)
# acc_abstr_1_3

## Phase 1+2+3: Abs + Kw
- Top-(K-1) 저널 추출 후, 유사 저널 1개 추가

In [351]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = True
tt_kw_s = s2

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K-1
    first_n = K-1
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    workers = -1
    with parallel_backend(backend='threading', n_jobs=workers):
        top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_kw_1_2_3 = accuracy_results.copy()
print(acc_abstr_kw_1_2_3)
acc_abstr_kw_1_2_3

Current K: 3


100%|████████████████████████████████████| 10307/10307 [00:37<00:00, 271.42it/s]


Current K: 5


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 299.07it/s]


Current K: 10


...

100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 301.78it/s]


Current K: 20


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 301.02it/s]


           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.4950 0.6316  0.7650  0.8275  0.8647
macro_acc 0.4093 0.5411  0.6882  0.7602  0.8142
mrr       0.3864 0.4309  0.4420  0.4359  0.4285


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.4950,0.6316,0.7650,0.8275,0.8647
macro_acc,0.4093,0.5411,0.6882,0.7602,0.8142
mrr,0.3864,0.4309,0.4420,0.4359,0.4285


In [353]:
import pandas as pd

# 결과 csv 저장
acc_abstr_kw_1_2_3.to_csv('./acc_model4/acc_abstr_kw_1_2_3.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_kw_1_2_3 = pd.read_csv('./acc_model4/acc_abstr_kw_1_2_3.csv', index_col=0)
# acc_abstr_kw_1_2_3

## Phase 1+2+3: Abs + Tt
- Top-(K-1) 저널 추출 후, 유사 저널 1개 추가

In [350]:
# Title and/or Keyword 여부에 따른 설정
tt_kw_exist = True
tt_kw_s = s3

accuracy_results = pd.DataFrame()

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K-1
    first_n = K-1
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs.tolist()

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    workers = -1
    with parallel_backend(backend='threading', n_jobs=workers):
        top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_tt_1_2_3 = accuracy_results.copy()
print(acc_abstr_tt_1_2_3)
acc_abstr_tt_1_2_3

Current K: 3


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 294.74it/s]


Current K: 5


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 296.63it/s]


Current K: 10


...

100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 294.84it/s]


Current K: 20


100%|████████████████████████████████████| 10307/10307 [00:34<00:00, 296.52it/s]


           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.4944 0.6306  0.7637  0.8256  0.8615
macro_acc 0.4093 0.5390  0.6852  0.7584  0.8095
mrr       0.3855 0.4252  0.4289  0.4158  0.4056


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.4944,0.6306,0.7637,0.8256,0.8615
macro_acc,0.4093,0.5390,0.6852,0.7584,0.8095
mrr,0.3855,0.4252,0.4289,0.4158,0.4056


In [352]:
import pandas as pd

# 결과 csv 저장
acc_abstr_tt_1_2_3.to_csv('./acc_model4/acc_abstr_tt_1_2_3.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_tt_1_2_3 = pd.read_csv('./acc_model4/acc_abstr_tt_1_2_3.csv', index_col=0)
# acc_abstr_tt_1_2_3

## Phase 1+2+3: Abs + Tt + Kw
- Top-(K-1) 저널 추출 후, 유사 저널 1개 추가

In [337]:
# 기존 모델에서 sum이 아닌 mean 사용한 모델 평가
# 1) 모든 유사도에 대한 mean (O)

tt_kw_exist = True
tt_kw_s = s23

accuracy_results = pd.DataFrame() # 정확도 결과 df

for K in K_list:
    print('Current K:', K)
    
    ## (1차) Test document별 Top-(K-1) 저널
    # Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
    # 1차에서 찾을 추천 후보 저널 수: K-1
    first_n = K-1
    top_k_journs = np.array(r)[:, :first_n]

    ## (2차) R-resort : List of Journals 재배치
    # Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
    if tt_kw_exist:
        r_resort = list()
        for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
            r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
    # 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
    else:
        r_resort = top_k_journs

    ## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
    #workers = os.cpu_count() * 2
    workers = -1
    with parallel_backend(backend='threading', n_jobs=workers):
        top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    
    
    ## 정확도 평가
    micro_acc = get_acc_micro(top_k_r, test_y) # Top-K Micro Accuracy (전체 Accuracy 합계 / 전체 N)
    macro_acc = get_acc_macro(top_k_r, test_y) # Top-K Macro Accuracy (저널별 평균 accuracy의 평균)
    mrr = get_mrr(top_k_r, test_y) # Top-K MRR (실제 저널이 추천 저널 리스트에 있다면 해당 순위를 반영해 점수 계산)
    top_k_eval = pd.DataFrame([micro_acc, macro_acc, mrr], index=['micro_acc', 'macro_acc', 'mrr'], columns=[f'Top-{K}'])
    
    accuracy_results = pd.concat([accuracy_results,top_k_eval], axis=1)
    
acc_abstr_tt_kw_1_2_3 = accuracy_results.copy()
print(acc_abstr_tt_kw_1_2_3)
acc_abstr_tt_kw_1_2_3

Current K: 3


100%|█████████████████████████████████████| 10307/10307 [02:54<00:00, 58.93it/s]


Current K: 5


100%|█████████████████████████████████████| 10307/10307 [02:32<00:00, 67.60it/s]


Current K: 10


...

Current K: 15


100%|█████████████████████████████████████| 10307/10307 [02:13<00:00, 76.99it/s]


Current K: 20


100%|█████████████████████████████████████| 10307/10307 [02:11<00:00, 78.10it/s]


,Top-3,Top-5,Top-10,Top-15,Top-20
micro_acc,0.4948,0.6319,0.7657,0.8270,0.8644
macro_acc,0.4091,0.5411,0.6886,0.7601,0.8143
mrr,0.3906,0.4411,0.4630,0.4591,0.4556


In [357]:
print(acc_abstr_tt_kw_1_2_3)

           Top-3  Top-5  Top-10  Top-15  Top-20
micro_acc 0.4948 0.6319  0.7657  0.8270  0.8644
macro_acc 0.4091 0.5411  0.6886  0.7601  0.8143
mrr       0.3906 0.4411  0.4630  0.4591  0.4556


In [354]:
import pandas as pd

# 결과 csv 저장
acc_abstr_tt_kw_1_2_3.to_csv('./acc_model4/acc_abstr_tt_kw_1_2_3.csv', index=True)

# 결과 csv 다시 읽기
# acc_abstr_tt_kw_1_2_3 = pd.read_csv('./acc_model4/acc_abstr_tt_kw_1_2_3.csv', index_col=0)
# acc_abstr_tt_kw_1_2_3

# # 사용자 평가용 Top-5 저널 출력
- 최종 프레임워크의 Top-5 추천 저널에 대한 사용자 평가

In [ ]:
K = 5

tt_kw_exist = True
tt_kw_s = s23

print('Current K:', K)

## (1차) Test document별 Top-(K-1) 저널
# Abstract에 대한 doc-to-doc 유사도 합계가 높은 순서로 저널 정렬됨
# 1차에서 찾을 추천 후보 저널 수: K-1
first_n = K-1
top_k_journs = np.array(r)[:, :first_n]

## (2차) R-resort : List of Journals 재배치
# Title and/or keyword가 있다면, test doc vs. train journ의 Title 및 Keyword 유사도 높은 순서로 재정렬
if tt_kw_exist:
    r_resort = list()
    for i in tqdm(range(len(tt_kw_s)), position=0, leave=True): # 10307
        r_resort.append(resort_r(tt_kw_s, top_k_journs, i))
# 없다면, Abstract 유사도 기준으로 정렬된 1차 추천 후보 리스트 그대로 사용
else:
    r_resort = top_k_journs

## (3차) 상위 1개 journal-journal 유사도 분석 및 추가 할 수 있는 후보 저널 탐색
#workers = os.cpu_count() * 2
workers = -1
with parallel_backend(backend='threading', n_jobs=workers):
    top_k_r = list(Parallel()(delayed(find_other_sim_journ)(top_journs_list) for top_journs_list in tqdm(r_resort, position=0, leave=True)))    

Current K: 5


100%|████████████████████████████████████| 10307/10307 [00:33<00:00, 303.39it/s]


In [ ]:
# 결과 Dataframe 정리
test_data_info = test[['title', 'abstract', 'journal']]
rec_top_5 = pd.DataFrame(top_k_r, columns=['R1','R2','R3','R4','R5'])

rec_result = pd.concat([test_data_info.reset_index(drop=True), rec_top_5], axis=1)#.rename(columns=)
rec_result = rec_result.rename(columns={'title':'제목','abstract':'초록','journal':'저널'})
rec_result

,제목,초록,저널,R1,R2,R3,R4,R5
0,충북지역 포도원 잡초방제별 생육 및 수량에 미치는 영향,본 시험은 충북지역 포도원 잡초방제별 생육 및 수량에 미치는 영향을 구명하기 위해 ...,농약과학회지,시설원예ㆍ식물공장,원예과학기술지,한국가금학회지,한국응용곤충학회지,농업과학연구
1,다중 추적자 영상을 위한 컴프턴 카메라,컴프턴 산란 현상을 이용하여 전자적 집속 방법으로 영상화하는 컴프턴카메라는 고민감도...,한국의학물리학회지:의학물리,한국의학물리학회지:의학물리,한국방사선학회논문지,한국정보통신학회논문지,한국음향학회지,대한방사선치료학회지
2,언어적인 항해안전정보 지원을 위한 의미해석 모델 구축에 관한 연구,"선박의 항해사가 안전 항해를 위해 GPS, ARPA, AIS, NAVTEX, VHF...",한국지능시스템학회논문지,한국항해항만학회지,디지털콘텐츠학회 논문지,한국정보통신학회논문지,한국지능시스템학회논문지,해양환경안전학회지
3,홍천 철-희토류 광상의 편마암질 주변암에 대한 SHRIMP U-Pb 연령측정,홍천에는 카보내타이트-포스코라이트 복합체로 구성된 철-희토류 광상이 분포한다. 이 ...,암석학회지,암석학회지,자원환경지질,한국광물학회지,한국제4기학회지,한국지구과학회지
4,중소기업의 FTA 특혜활용을 위한 HS 품목분류 자가결정 방법에 대한 연구,FTA가 확대됨에 따라 FTA별로 상이한 원산지결정기준을 활용한 특혜 수혜가 점차 ...,통상정보연구,통상정보연구,한국항만경제학회지,유통과학연구,한국유통학회지:유통연구,경영과정보연구
...,...,...,...,...,...,...,...,...
10302,가공된 순환자원을 시멘트 혼화재로 활용한 흙 시멘트 공시체의 강도 특성에 관한 연구,"본 연구에서는 소성 굴패각, 비소성 굴패각, 자력선별된 전로 제강슬래그와 Fly a...",터널과지하공간,한국건설순환자원학회논문집,한국구조물진단유지관리공학회 논문집,한국콘크리트학회논문집,대한토목학회논문집,한국건축시공학회지
10303,E-Shape 강재이력댐퍼의 수치모델과 기초격리구조물의 지진응답,"최근 대규모의 지진피해로 인해 내진설계에 대한 관심이 높아지면서, LRB(Lead ...",대한토목학회논문집,한국전산구조공학회논문집,대한토목학회논문집,한국구조물진단유지관리공학회 논문집,한국콘크리트학회논문집,대한기계학회논문집A
10304,초등학교 5학년 영어 교과서 분석: 어휘학습전략 중심으로,최근 외국어 학습에 있어 어휘 학습의 중요성이 부각되며 어휘학습전략 교육에 대한 관...,한국콘텐츠학회논문지,한국초등수학교육학회지,대한수학교육학회지:수학교육학연구,한국학교수학회논문집,한국콘텐츠학회논문지,한국수학교육학회지시리즈C:초등수학교육
10305,도계 후 원료육의 저장기간 동안 가슴육의 품질 특성,본 연구는 도계 후 원료육의 저장기간에 따른 품질 특성을 조사하여 가공업소에서 반입...,한국가금학회지,원예과학기술지,한국가금학회지,시설원예ㆍ식물공장,대한환경공학회지,농업과학연구


In [ ]:
# csv 저장
rec_result.to_csv('top5_result.csv', encoding='utf-8-sig')

# rec_result = pd.read_csv('top5_result.csv', index_col=0)

# 끝